# Hidden markov model generalized to 2 parenting relationship (also ngram model, n=3)

In [135]:
# P(X_i|X_i-1,X_i-2)

In [136]:
import nltk
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle 
import json
import os
from pathlib import Path
from IPython.display import Image, Audio
from music21 import note , chord , stream , instrument , converter   
import mido

In [137]:
# create a dict with the ngram model, it receives a list with the samples as string
ngram_dict = {}

def create_trigram_dict(corpus):
    n = 3
    ngrams = nltk.ngrams(corpus, n)
    
    for grams in ngrams:
        dict_key = grams[:-1][0] + " " + grams[:-1][1]
        if dict_key in ngram_dict:
            ngram_dict[dict_key].append(grams[-1])
        else:
            ngram_dict[dict_key] = []
            ngram_dict[dict_key].append(grams[-1])

In [138]:
def generate_trigram(seed, samples, ngram_dict):
    output = seed  
    for i in range(samples):
        # When it reaches the last prefix, there is no suffix, so end
        try:
            new_sample = random.choice(ngram_dict[seed])
        except:
            return output
        output += " " + new_sample
        seed = seed.split(" ")[1] + " " + new_sample

    return output

In [139]:
songs = []
for filename in os.listdir("dataset/midi_songs"):
    if filename.endswith(".mid"): 
        #print(filename)
        s = 'dataset/midi_songs/'+filename
        #song = mido.MidiFile(s)
        conv = converter.parse(s)
        songs.append(conv)
        continue
    else:
        continue


In [140]:
notes = []
for s in songs:
    notes_to_parse = s.flat.notes
    notes.append(notes_to_parse)
    

In [141]:
notes[:5]

[<music21.stream.iterator.StreamIterator for Score:0x2d0006910 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x2986c51f0 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x2cffb0880 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x2a7c81670 @:0>,
 <music21.stream.iterator.StreamIterator for Score:0x2a4159d60 @:0>]

In [142]:
#                                      song1  song2
#generate list of list of notes --> [[A,F,C],[D,G,B]]
listoflists = []
for n in notes:
    notes_demo = []
    for element in n:
        # if the element is a Note , then store it's Pitch
        if isinstance(element , note.Note):
            notes_demo.append(str(element.pitch))
        
        elif isinstance(element , chord.Chord):
            notes_demo.append('+'.join(str(n) for n in element.normalOrder))
            
    listoflists.append(notes_demo)
        

In [143]:
# converting into a single string each song in listoflists
final = []
for song in listoflists:
    l = song
    melody = ''
    for i in l:
        melody = melody + ' ' + i
    final.append(melody)
    melody = ''

In [144]:
dictionaries = []
for progression in final:
    ngram_dict = {}
    create_trigram_dict(progression.split(" "))
    dictionaries.append(ngram_dict)
    

In [230]:
dictionaries[6]

{' A4': ['D4'],
 'A4 D4': ['B-4', 'F4', 'D5'],
 'D4 B-4': ['F4', 'B-3'],
 'B-4 F4': ['F5'],
 'F4 F5': ['C4', 'E5', 'E5'],
 'F5 C4': ['G4', 'E4'],
 'C4 G4': ['E4', 'E4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5'],
 'G4 E4': ['A4', 'A4'],
 'E4 A4': ['B-3', 'B-3', 'C4', 'D5', 'D5'],
 'A4 B-3': ['E5', 'D4', 'B-4'],
 'B-3 E5': ['D4', 'D4'],
 'E5 D4': ['F4', 'G4', 'G4', 'B-3', 'A3'],
 'D4 F4': ['A3',
  'A4',
  'D4',
  'A4',
  'E5',
  'E5',
  'E5',
  'A4',
  'C5',
  'D4',
  'D4',
  'E5',
  'E5',
  'E5',
  'A4',
  'C5',
  'D4',
  'D4'],
 'F4 A3': ['C5'],
 'A3 C5': ['C4'],
 'C5 C4': ['D5'],
 'C4 D5': ['G3', 'D3', 'B-3', 'B-3'],
 'D5 G3': ['E4', 'C5', 'E5'],
 'G3 E4': ['B-3'],
 'E4 B-3': ['F4'],
 'B-3 F4': ['D4'],
 'F4 D4': ['C5',
  'A3',
  'B-3',
  'B-3',
  'G5',
  'G5',
  'C4',
  'B-3',
  'B-3',
  'G5',
  'G5',
  'F5'],
 'D4 C5': ['0+4'],
 'C5 0+4': ['A2'],
 '0+4 A2': ['A4'],
 'A2 A4': ['D4'],
 'F4 A4': ['F4', 'D4', 'B-5', 'G5', 'B-5', 'E5', 'B-5', 'G5', 'B-5', 'E5'],
 'A4 F4': ['E5'],
 'F4 E5': ['C4

In [237]:
#generate melody based on trigram model and pattern recognition on ngram_dict
generated_melody = generate_trigram("B-3 D4", 100, dictionaries[6])
print(generated_melody)

B-3 D4 F4 C5 A3 E4 A4 D5 D4 F4 E5 C4 E4 C4 9+0+4 A5 E6 C6 D6 B-3 D4 F4 E5 C4 E4 F5 G4 G5 A5 F3 F2 5+9 9+0 G2 1+7 A2 11+2 B2 0+4 C#3 2+5+9 B-3 D4 F4 E5 C4 E4 C5 G4 C5 A5 F4 A4 B-5 C5 A5 B-3 D4 G5 F4 F5 E5 C2 2+5+9 D2 G5 F5 D4 F5 F4 D4 B-3 D4 B-4 F4 F5 E5 C4 E4 C4 9+0+4 A5 E6 C6 D6 C4 E6 E4 F6 E6 D3 A3 D4 E4 F4 E4 C3 F4 G4 A4 F3 G4


In [238]:
#create suitable format for playing midi file concatenating the string with the notation and tempo
generated_melody = "tinynotation: 4/4" + generated_melody

In [239]:
littleMelody = converter.parse(generated_melody)
littleMelody.show('midi')

In [240]:
#saving
littleMelody.write("midi", "testaudio.mkv")

'testaudio.mkv'